# Problem set 3
## Exercise 5

---

&nbsp;

### Imports:

&nbsp;

### Task 1:

> Load the trajectories and perform MC evaluation for $v_{\pi}(s)$.

In [22]:
import numpy as np
from mountain.envs import MountainEnv

# Initialize Mountain environment (v1 / Mountain Lite)
env = MountainEnv()
state = env.reset()  # starting at row 15, col 0 (near top-left)


In [23]:
import pickle
with open("trajectories.pickle", "rb") as f:
    trajectories = pickle.load(f)
print(f"Loaded {len(trajectories)} trajectories from SpaceY's policy π.")
# (Each trajectory is a list of (state, action, reward) tuples for one episode)

Loaded 1000 trajectories from SpaceY's policy π.


In [24]:
import numpy as np

# Initialize action-value table Q and visit counts
n_actions = 5
Q = np.zeros((env.height, env.width, n_actions))
returns_count = np.zeros((env.height, env.width, n_actions))

def epsilon_greedy_action(state, Q, epsilon):
    """Select an action using epsilon-greedy strategy based on Q."""
    r, c = state
    if np.random.rand() < epsilon:
        # Exploration: random action
        return np.random.randint(0, n_actions)
    else:
        # Exploitation: choose best known action (break ties randomly)
        values = Q[r, c]
        max_val = np.max(values)
        best_actions = np.flatnonzero(np.isclose(values, max_val))
        return np.random.choice(best_actions)

# Monte Carlo Control parameters
num_episodes = 10000
min_epsilon = 0.1  # minimum exploration rate
epsilon = 1.0      # start fully random

for episode in range(1, num_episodes+1):
    # Decay epsilon over time (ensure eventually policy is greedy in the limit)
    # For example, linear decay:
    epsilon = max(min_epsilon, 1 - episode/num_episodes)  
    episode_experience = []  # to store (state, action, reward) tuples
    
    # Generate one episode
    state = env.reset()
    done = False
    while not done:
        action = epsilon_greedy_action(state, Q, epsilon)
        next_state, reward, done = env.step(action)
        episode_experience.append((state, action, reward))
        state = next_state
    
    # Calculate returns backwards and update Q-values
    G = 0.0
    visited = set()  # track state-action pairs visited in this episode
    # Traverse episode in reverse to compute returns
    for state, action, reward in reversed(episode_experience):
        G += reward  # accumulate return
        if (state, action) not in visited:  # first-visit
            visited.add((state, action))
            r, c = state
            returns_count[r, c, action] += 1
            # Incremental MC update toward return G
            Q[r, c, action] += (G - Q[r, c, action]) / returns_count[r, c, action]
    # (Policy implicitly improves as Q is updated and epsilon-greedy uses new Q)

ValueError: too many values to unpack (expected 3)